In [13]:
!pip install sympy==1.12


   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   -------------- ------------------------- 2.1/5.7 MB 16.8 MB/s eta 0:00:01
   -------------------------------------- - 5.5/5.7 MB 14.0 MB/s eta 0:00:01
   ---------------------------------------- 5.7/5.7 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.7.0.dev20250113+cu126 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.


In [5]:
#!pip install torch
!pip install scgpt "flash-attn<1.0.5"
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/203.0 MB 5.6 MB/s eta 0:00:37
   ---------------------------------------- 2.1/203.0 MB 4.9 MB/s eta 0:00:42
    --------------------------------------- 3.1/203.0 MB 5.0 MB/s eta 0:00:41
    --------------------------------------- 4.5/203.0 MB 5.3 MB/s eta 0:00:38
   - -------------------------------------- 5.5/203.0 MB 5.4 MB/s eta 0:00:37
   - -------------------------------------- 6.6/203.0 MB 5.2 MB/s eta 0:00:38
   - -------------------------------------- 7.9/203.0 MB 5.5 MB/s eta 0:00:36
   - -------------------------------------- 9.4/203.0 MB 5.7 MB/s eta 0:00:34
   -- ------------------------------------- 10.5/203.0 MB 5.7 MB/s eta 0:00:34
   -- ------------------------------------- 11.3/203.0 MB 5.5 MB/s eta 0:00:35
   -- -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Using cached scgpt-0.1.5-py3-none-any.whl.metadata (7.9 kB)
  Using cached flash_attn-1.0.4.tar.gz (2.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [22 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\gaiacronus\AppData\Local\Temp\pip-install-43yljevc\flash-attn_77605d91efbd4740a8193cf084675b72\setup.py", line 106, in <module>
      raise_if_cuda_home_none("flash_attn")
    File "C:\Users\gaiacronus\AppData\Local\Temp\pip-install-43yljevc\flash-attn_77605d91efbd4740a8193cf084675b72\setup.py", line 53, in raise_if_cuda_home_none
      raise RuntimeError(
  RuntimeError: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
  
   If your intention is to cross-compile, this is not an error.
  By default, Apex wi

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip show torch torchtext

Name: torch
Version: 2.5.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: C:\Users\gaiacronus\AppData\Roaming\Python\Python312\site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: torchaudio, torchtext, torchvision
---
Name: torchtext
Version: 0.18.0
Summary: Text utilities, models, transforms, and datasets for PyTorch.
Home-page: https://github.com/pytorch/text
Author: PyTorch Text Team
Author-email: packages@pytorch.org
License: BSD
Location: C:\Users\gaiacronus\.conda\envs\demo3\Lib\site-packages
Requires: numpy, requests, torch, tqdm
Required-by: 


In [1]:
import scanpy as sc
import TOSICA
import os
os.environ["PYTORCH_SYMBOLIC_SHAPE_INFERENCE"] = "0"
from scgpt.tokenizer import tokenize_and_pad
from scgpt.model import load_pretrained_scGPT
from scgpt.preprocess import prepare_for_training
# Load the h_pancreas datasets
train_path = "C:/Users/gaiacronus/Downloads/work/combine/adata/hPancreas_train_adata.h5ad"  # Training dataset
test_path = "C:/Users/gaiacronus/Downloads/work/combine/adata/hPancreas_test_adata.h5ad"   # Testing dataset

# Load the datasets as AnnData objects
train_adata = sc.read_h5ad(train_path)
test_adata = sc.read_h5ad(test_path)

#  Preprocess the datasets
# Align the gene names in both datasets
common_genes = train_adata.var_names.intersection(test_adata.var_names)
train_adata = train_adata[:, common_genes]
test_adata = test_adata[:, common_genes]
# Convert AnnData object to scGPT-compatible format
train_scGPT = prepare_for_training(train_adata)
test_scGPT = prepare_for_training(test_adata)
# Normalize the data 
sc.pp.normalize_total(train_adata, target_sum=1e4)
sc.pp.log1p(train_adata)

sc.pp.normalize_total(test_adata, target_sum=1e4)
sc.pp.log1p(test_adata)
ref_adata=train_adata
query_adata=test_adata
label_key = "Celltype2"  
gmt_path = "C:/Users/gaiacronus/Downloads/work/combine/TOSICA/resources/immune.gmt"  # Path to the gene set file 
project_name = "tostry"    

# Train the model
TOSICA.train(
    ref_adata,
    gmt_path=gmt_path,
    project=project_name,
    epochs=3,
    label_name=label_key
)

 #Predict cell types in the test dataset
model_weight_path = f"{project_name}/model_weight.h5"

# Loading pre-trained scGPT model and fine-tune 
model = load_pretrained_scGPT(best_model.pt)  
#  Fine-tune the model using the training data
model.train(train_scGPT, epochs=3, batch_size=32)

# Predict cell types
predicted_adata = TOSICA.pre(
    query_adata,
    model_weight_path=model_weight_path,
    project=project_name
)

predictions = model.predict(test_scGPT)
test_adata.obs["scGPT_predictions"] = predictions
 #Inspect the predictions
# Predicted cell types and probabilities are added to the `obs` of the AnnData object
print(predicted_adata.obs.head())  # Check the predicted labels and probabilities

# Save the results for further analysis
predicted_adata.write("h_pancreas_predicted.h5ad")

#  Compare TOSICA and scGPT results
print("TOSICA Predictions:\n", predicted_adata.obs.head())
print("scGPT Predictions:\n", test_adata.obs["scGPT_predictions"].head())


C:\Users\gaiacronus\Downloads\work\combine\scgpt\model\model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
C:\Users\gaiacronus\Downloads\work\combine\scgpt\model\multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
C:\Users\gaiacronus\.conda\envs\demo3\Lib\site-packages\torchtext\__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: [WinError 127] The specified procedure could not be found